In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import polars as pl

In [47]:
df = pd.read_parquet('data/output/raw_chunks/chunk_0_4999.parquet')

In [53]:
df

,rs#,alleles,chrom,pos,strand,assembly#,center,protLSID,assayLSID,panelLSID,...,IRIS_313-15902,IRIS_313-15903,IRIS_313-15904,IRIS_313-15905,IRIS_313-15906,IRIS_313-15907,IRIS_313-15908,IRIS_313-15909,IRIS_313-15910,NB
0,chr1_1384,G/T,chr1,1384,+,NaN,NaN,NaN,NaN,NaN,...,GG,GG,NN,GG,GG,GG,GG,GG,GG,GG
1,chr1_1399,C/A,chr1,1399,+,NaN,NaN,NaN,NaN,NaN,...,CC,CC,NN,CC,CC,CC,CC,CC,CC,CC
2,chr1_1409,C/T,chr1,1409,+,NaN,NaN,NaN,NaN,NaN,...,CC,CC,NN,CC,CC,CC,CC,CC,CC,CC
3,chr1_1427,A/T,chr1,1427,+,NaN,NaN,NaN,NaN,NaN,...,AA,AA,NN,AA,AA,AA,AA,AA,AA,AA
4,chr1_1429,C/T,chr1,1429,+,NaN,NaN,NaN,NaN,NaN,...,CC,CC,NN,CC,CC,CC,CC,CC,CC,CC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,chr1_150193,C/T,chr1,150193,+,NaN,NaN,NaN,NaN,NaN,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
4996,chr1_150201,G/A,chr1,150201,+,NaN,NaN,NaN,NaN,NaN,...,GG,GG,GG,GG,GG,GG,GG,GG,GG,GG
4997,chr1_150275,A/G,chr1,150275,+,NaN,NaN,NaN,NaN,NaN,...,AA,AA,AA,AA,AA,AA,AA,AA,AA,AA
4998,chr1_150280,C/A,chr1,150280,+,NaN,NaN,NaN,NaN,NaN,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC


In [11]:
from string import ascii_letters, digits, printable, punctuation
len(ascii_letters + digits + punctuation )

94

In [2]:
from adn.models.tokenizer import get_one_snp_tokenizer


tokenizer = get_one_snp_tokenizer()
tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=68, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<BOS>', 'eos_token': '<EOS>', 'unk_token': '<UNK>', 'cls_token': '<CLS>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<BOS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<EOS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<UNK>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<CLS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [7]:
from pathlib import Path
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors
from adn.models.tokenizer import BOS_TOKEN_STR, UNK_TOKEN_STR, EOS_TOKEN_STR, CLS_TOKEN_STR

def get_one_snp_bpe_tokenizer(tokenizer_path: Path) -> PreTrainedTokenizerFast:
    """
    Charge un tokenizer BPE sauvegardé au format JSON et le wrappe
    dans un PreTrainedTokenizerFast pour compatibilité HuggingFace.
    """
    tokenizer = Tokenizer.from_file(str(tokenizer_path))

    tokenizer.pre_tokenizer = tokenizer.pre_tokenizer or processors.Whitespace()  # sécurité

    bos_token_id = tokenizer.token_to_id(BOS_TOKEN_STR)
    eos_token_id = tokenizer.token_to_id(EOS_TOKEN_STR)
    cls_token_id = tokenizer.token_to_id(CLS_TOKEN_STR)

    tokenizer.post_processor = processors.TemplateProcessing(
        single=f"{BOS_TOKEN_STR} $A {CLS_TOKEN_STR}",
        special_tokens=[
            (cls_token_id, CLS_TOKEN_STR),
            (bos_token_id, BOS_TOKEN_STR),
            (eos_token_id, EOS_TOKEN_STR),
        ],
    )

    wrapped_tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=tokenizer,
        unk_token=UNK_TOKEN_STR,
        cls_token=CLS_TOKEN_STR,
        bos_token=BOS_TOKEN_STR,
        eos_token=EOS_TOKEN_STR,
    )

    return wrapped_tokenizer

tokenizer_path = Path('output/tokenizer/tokenizer/tokenizer.json')
tokenizer = get_one_snp_bpe_tokenizer(tokenizer_path)
tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=44, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<BOS>', 'eos_token': '<EOS>', 'unk_token': '<UNK>', 'cls_token': '<CLS>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<UNK>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<BOS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<EOS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<CLS>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [68]:
ref = pd.read_parquet("data/output/all_main_alleles.parquet")
ref["main_allele"].value_counts()

main_allele
CC    369704
GG    367131
TT    182795
AA    182518
Name: count, dtype: int64

In [60]:
df["alleles"].value_counts()

alleles
C/T    1412
G/A    1355
T/C     460
A/G     422
G/T     274
C/A     264
A/T     178
T/A     174
C/G     131
G/C     121
T/G     116
A/C      93
Name: count, dtype: int64

In [74]:
df2 = pl.read_parquet('data/output/all_main_allele.parquet')
df2 = df2.sort('position')
df2

position,main_allele
i64,str
1384,"""GG"""
1399,"""CC"""
1409,"""CC"""
1427,"""AA"""
1429,"""CC"""
…,…
43269742,"""GG"""
43269856,"""AA"""
43269879,"""AA"""


In [66]:
df2["position"].diff().describe().round(2)

count    1107867.00
mean          39.06
std          655.61
min            1.00
25%            5.00
50%           12.00
75%           29.00
max       432274.00
Name: position, dtype: float64

In [68]:
desc = df2[100:250]["position"].describe()
start = int(desc["min"])
end = int(desc["max"])
print(f"Start: {start}, End: {end}")

Start: 1974, End: 4732


In [69]:
df3 = pl.read_parquet('data/output/SNPs/B001.parquet').to_pandas()

In [70]:
df3[(df3["position"] >= start) & (df3["position"] <= end)]

,allele,main_allele,position,individual


In [78]:
df2[100:250]

position,main_allele
i64,str
1974,"""GG"""
1977,"""GG"""
1979,"""TT"""
1986,"""AA"""
1989,"""GG"""
…,…
4629,"""TT"""
4658,"""TT"""
4675,"""GG"""


,allele,main_allele,position,individual
0,GT,GG,6918,B001
1,GT,GG,6946,B001
2,CC,AA,17263,B001
3,AG,AA,21665,B001
4,TG,TT,21680,B001
...,...,...,...,...
43797,TT,CC,43244177,B001
43798,AA,AC,43253999,B001
43799,AA,GG,43254132,B001
43800,GG,GA,43254300,B001


In [37]:
ref_df = pl.read_parquet("data/output/all_main_alleles.parquet")
ref_df = ref_df.drop(["__index_level_0__"])
individual_df = pl.read_parquet("data/output/SNPs/B001.parquet")

In [38]:
sub_ref = ref_df[0:5000]
start_pos = sub_ref["position"][0]
end_pos = sub_ref["position"][-1]
print(f"Start: {start_pos}, End: {end_pos}")
# Filter the individual_df DataFrame to get the subset of individuals



Start: 1384, End: 152418


In [42]:
sub_individual = individual_df.filter(
    (individual_df["position"] >= start_pos) & (individual_df["position"] <= end_pos)
)

sub_ref_updated = sub_ref.join(
    sub_individual[["allele", "position"]],
    on="position",
    how="left"
)

sub_ref_updated = sub_ref_updated.with_columns(
    pl.col("allele").fill_null(pl.col("main_allele"))
)

sub_ref_updated.filter(
    (pl.col("main_allele") != pl.col("allele"))
)

position,main_allele,allele
i64,str,str
6918,"""GG""","""GT"""
6946,"""GG""","""GT"""
17263,"""AA""","""CC"""
21665,"""AA""","""AG"""
21680,"""TT""","""TG"""
78832,"""GG""","""GA"""
129602,"""CC""","""TT"""


In [29]:
sub_individual

allele,main_allele,position,individual
str,str,i64,str
"""GT""","""GG""",6918,"""B001"""
"""GT""","""GG""",6946,"""B001"""
"""CC""","""AA""",17263,"""B001"""
"""AG""","""AA""",21665,"""B001"""
"""TG""","""TT""",21680,"""B001"""
"""GA""","""GG""",78832,"""B001"""
"""TT""","""CC""",129602,"""B001"""
